# 0. Import required libraries

###  managed to loop the code for the different type of vessels, so you can now see all the values given in table 7 of van Dosser et al. 2020. No adjustment of the calculations itself but more to get an overview of all the vessels. The benefit of the code now is that you can play with the actual depth based on your own simulation and determine the DWT or payload for that situation. Example given a depth of 2.5 meters.--Frederik

In [1]:
import numpy as np
import pandas as pd

## 1. Regression coefficients

In [2]:
#Linear regression coefficients for the Capacity Index model (Table 3 in paper Van Dorsser et al (2020))
CI_coefs = dict({"intercept": 2.0323139721 * 10**1,
                
                "c1": -7.8577991460 * 10**1,
                "c2": -7.0671612519 * 10**0,
                "c3": 2.7744056480 * 10**1,
                "c4": 7.5588609922 * 10**-1,
                "c5": 3.6591813315 * 10**1
                })

#Linear regression coefficients for the Design Draft model (Table 5 in paper Van Dorsser et al (2020))

Tdesign_coefs = dict({"intercept":0,
                     "c1": 1.7244153371,
                     "c2": 2.2767179246,
                     "c3": 1.3365379898,
                     "c4": -5.9459308905,
                     "c5": 6.2902305560*10**-2,
                     "c6": 7.7398861528*10**-5,
                     "c7": 9.0052384439*10**-3,
                     "c8": 2.8438560877
                     })

#Linear regression coefficients for the Empty Draft model (Table 4 in paper Van Dorsser et al (2020))

Tempty_coefs = dict({"intercept": 7.5740820927*10**-2,
                    "c1": 1.1615080992*10**-1,
                    "c2": 1.6865973494*10**-2,
                    "c3": -2.7490565381*10**-2,
                    "c4": -5.1501240744*10**-5,
                    "c5": 1.0257551153*10**-1, # Dummy DH Dry Bulk & DH Container
                    "c6": 2.4299435211*10**-1, # Dummy DH Tanker 
                    "c7": -2.1354295627*10**-1, # Dumb Barge
                    })

#Linear regresion coefficients for Design Capacity (DWT) model

capacity_coefs = dict({"intercept": -1.6687441313*10**1,
                     "c1": 9.7404521380*10**-1,
                     "c2": -1.1068568208,
                      })

### 2. Capacity index
Input:
- T_empty = Empty draft
- T_actual = Actual draft (maximum possible draft, taking ukc into account)

In [3]:
def cap_index(T_empty, T_actual):
    CI = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2) \
    + (CI_coefs["c3"] * T_actual) + (CI_coefs["c4"] * T_actual**2) \
    + (CI_coefs["c5"] * (T_empty * T_actual))
    
    return CI

### 3. Design draft

Linear regression model to estimate the design draft when this parameter is not known.
Input:

- vessel = String of the vessel type (Container, Dry, Barge, Tanker)
- L = Length of the vessel
- B = Beam of the vessel

In [4]:
def design_draft(vessel, L, B):
    
    if vessel == "Container":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [1,0,0,0]
        
    elif vessel == "Dry_SH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
        
    elif vessel == "Dry_DH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
        
    elif vessel == "Barge":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,1,0]
        
    elif vessel == "Tanker":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,1]
    
    T_design = Tdesign_coefs['intercept'] \
    + (dum_container * Tdesign_coefs['c1']) + (dum_dry * Tdesign_coefs['c2']) + (dum_barge * Tdesign_coefs['c3']) + (dum_tanker * Tdesign_coefs['c4']) \
    + (Tdesign_coefs['c5'] * dum_container * L**0.4 * B**0.6) + (Tdesign_coefs['c6'] * dum_dry * L**0.7 * B**2.6) \
    + (Tdesign_coefs['c7'] * dum_barge * L**0.3 * B**1.8) + (Tdesign_coefs['c8'] * dum_tanker * L**0.1 * B**0.3)
   
    return T_design

### 4. Empty draft

Linear regression model to estimate the empty draft when this parameter is not known.
Input:

- vessel = String of the vessel type (Container, Dry_SH, Dry_DH, Barge, Tanker)
- T_design = Design Draft
- L = Length of the vessel
- B = Beam of the vessel

In [5]:
def empty_draft(vessel, T_design, L, B):
    
    if vessel == "Container":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [1,0,0,0]
        
    elif vessel == "Dry_SH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,0]
    
    elif vessel == "Dry_DH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
    
    elif vessel == "Barge":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,1,0]
        
    elif vessel == "Tanker":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,1]
    
    T_empty = Tempty_coefs['intercept'] \
    + (Tempty_coefs['c1'] * B) + (Tempty_coefs['c2'] * ((L * T_design) / B)) + (Tempty_coefs['c3'] * (np.sqrt(L * B))) \
    + (Tempty_coefs['c4'] * (L * B * T_design)) + (Tempty_coefs['c5'] * dum_container) + (Tempty_coefs['c5'] * dum_dry) \
    + (Tempty_coefs['c6'] * dum_tanker) + (Tempty_coefs['c7'] * dum_barge)
    
    return T_empty

### 5. Design capacity

Linear regression model to estimate the maximum capacity (DWT) when this parameter is not known.
Input:

- L = Length of the vessel
- B = Beam of the vessel

In [6]:
def design_capacity(T_design, T_empty, L, B):
    
    DWT = capacity_coefs['intercept'] + (capacity_coefs['c1'] * L * B * T_design) \
    + (capacity_coefs['c2'] * L * B * T_empty)
    
    return DWT

### 6. Actual draft

The actual draft is the draft which is maximum possible taking ukc and the water depth into account.

- T_design = Design draft
- T_empty = Empty draft
- depth = minimum available water depth
- ukc - under keel clearance (depends on river bed, cargo type and shippers choice

In [7]:
def actual_draft(T_design, T_empty, depth, ukc):
    
    if (T_design <= depth-ukc):
        
        T_actual = T_design
        
    elif T_empty > (depth -ukc):
        
        T_actual =  (f"No trip possible. Available depth smaller than empty draft: {depth - T_empty} m")
    
    elif (T_design > depth):
        
        T_actual = depth - ukc
        
    
    
    return T_actual

### 7. Application

### 7.1 Calculation of tankers

In [8]:
Tankers = {'type': ['Tanker','Tanker','Tanker'],
        'CEMT-class':  ['IV','V','VI'],
        'Length':[85, 110, 135],
        'Width': [9.50, 11.40, 17.50],
        'draft design':[2.77, 3.50, 5.02], #Deep water desing draft
        'draft mod':[1.30, 1.40, 1.50]} # Minimum operational draft

Tankers_df = pd.DataFrame (Tankers, columns = ['type','CEMT-class','Length','Width','draft design', 'draft mod'])

#print(Tankers_df)

In [9]:
consumable_design=0.06
consumable_lowwater=0.04
ukc=0.2

for index, row in Tankers_df.iterrows():
    T_design = design_draft(row['type'], row['Length'], row['Width'])
    T_empty = empty_draft(row['type'], T_design, row['Length'], row['Width'])
    depth1=row['draft mod']+ukc
    depth2=T_design+ukc
    T_actual_1 = actual_draft(T_design, T_empty, depth=2.5, ukc=0.2)
    T_actual_2 = actual_draft(T_design, T_empty, depth=depth2, ukc=0.2)
    CI_1 = cap_index(T_empty, T_actual=T_actual_1) #Capacity index at depth1
    CI_2 = cap_index(T_empty, T_actual=T_actual_2) #Capacity in dex at depth2 (maximum load factor)
    DWT_dc = design_capacity(T_design, T_empty, row['Length'], row['Width']) #Calculate the design capacity
    mod_capacity=DWT_dc*CI_1/CI_2 # Calculate the capacity for minimum operaional draft based on the design capacity of the vessel type
    payload_design = (1-consumable_design)*DWT_dc
    payload_lowwater = mod_capacity-consumable_lowwater*DWT_dc
    print(round(T_design,2),round(DWT_dc,0), round(payload_design),row['draft mod'], round(mod_capacity), round(payload_lowwater))

2.77 1316.0 1237 1.3 972 919
3.5 2849.0 2679 1.4 1456 1342
5.02 8759.0 8233 1.5 2680 2329


### 7.2 Calculation for Dry bulk Singel Hull vessels

In [10]:
Dry_bulk = {'type': ['Dry_SH','Dry_SH','Dry_SH','Dry_SH','Dry_SH','Dry_DH','Dry_DH'],
        'CEMT-class':  ['II', 'III','IV','V','IV','V','VI'],
        'Length': [55, 80, 85, 110, 85, 110, 135],
        'Width': [6.00, 8.20, 9.50, 11.40, 9.50, 11.40, 11.40],
        'draft design':[2.41, 2.67, 2.88, 3.44, 2.88, 3.44, 3.62], #Deep water desing draft
        'draft mod':[1.20, 1.20, 1.30, 1.40, 1.30, 1.40, 1.50]} # Minimum operational draft

df = pd.DataFrame (Dry_bulk, columns = ['type','CEMT-class','Length','Width','draft design', 'draft mod'])

print (df)

     type CEMT-class  Length  Width  draft design  draft mod
0  Dry_SH         II      55    6.0          2.41        1.2
1  Dry_SH        III      80    8.2          2.67        1.2
2  Dry_SH         IV      85    9.5          2.88        1.3
3  Dry_SH          V     110   11.4          3.44        1.4
4  Dry_SH         IV      85    9.5          2.88        1.3
5  Dry_DH          V     110   11.4          3.44        1.4
6  Dry_DH         VI     135   11.4          3.62        1.5


In [11]:
consumable_design=0.06
consumable_lowwater=0.04
ukc=0.1

for index, row in df.iterrows():
    T_design = design_draft(row['type'], row['Length'], row['Width'])
    T_empty = empty_draft(row['type'], T_design, row['Length'], row['Width'])
    depth1=row['draft mod']+ukc
    depth2=T_design+ukc
    T_actual_1 = actual_draft(T_design, T_empty, depth=depth1, ukc=0.1)
    T_actual_2 = actual_draft(T_design, T_empty, depth=depth2, ukc=0.1)
    CI_1 = cap_index(T_empty, T_actual=T_actual_1) #Capacity index at depth1
    CI_2 = cap_index(T_empty, T_actual=T_actual_2) #Capacity in dex at depth2 (maximum load factor)
    DWT_dc = design_capacity(T_design, T_empty, row['Length'], row['Width']) #Calculate the design capacity
    mod_capacity=DWT_dc*CI_1/CI_2 # Calculate the capacity for minimum operaional draft based on the design capacity of the vessel type
    payload_design = (1-consumable_design)*DWT_dc
    payload_lowwater = mod_capacity-consumable_lowwater*DWT_dc
    print(round(T_design,2),round(DWT_dc,0), round(payload_design),row['draft mod'], round(mod_capacity), round(payload_lowwater))

2.41 537.0 505 1.2 176 155
2.67 1202.0 1130 1.2 309 261
2.88 1612.0 1516 1.3 422 358
3.44 3125.0 2937 1.4 710 585
2.88 1612.0 1516 1.3 422 358
3.44 2982.0 2803 1.4 588 469
3.62 3944.0 3707 1.5 874 716


### 7.3 Calculation for containervessels

In [12]:
Containervessels = {'type': ['Container','Container','Container','Container','Container'],
        'CEMT-class':  ['III','IV','V','VI','VI+'],
        'Length': [63, 85, 110, 135, 135],
        'Width': [7.00, 9.50, 11.45, 14.25, 17.50],
        'draft design':[2.78, 3.16, 3.50, 3.93, 4.22], #Deep water desing draft
        'draft mod':[1.20, 1.30, 1.40, 1.50, 1.50]} # Minimum operational draft

container_df = pd.DataFrame (Containervessels, columns = ['type','CEMT-class','Length','Width','draft design', 'draft mod'])

print(container_df)

        type CEMT-class  Length  Width  draft design  draft mod
0  Container        III      63   7.00          2.78        1.2
1  Container         IV      85   9.50          3.16        1.3
2  Container          V     110  11.45          3.50        1.4
3  Container         VI     135  14.25          3.93        1.5
4  Container        VI+     135  17.50          4.22        1.5


In [13]:
consumable_design=0.06
consumable_lowwater=0.04
ukc=0.2

for index, row in container_df.iterrows():
    T_design = design_draft(row['type'], row['Length'], row['Width'])
    T_empty = empty_draft(row['type'], T_design, row['Length'], row['Width'])
    depth1=row['draft mod']+ukc
    depth2=T_design+ukc
    T_actual_1 = actual_draft(T_design, T_empty, depth=depth1, ukc=0.2)
    T_actual_2 = actual_draft(T_design, T_empty, depth=depth2, ukc=0.2)
    CI_1 = cap_index(T_empty, T_actual=T_actual_1) #Capacity index at depth1
    CI_2 = cap_index(T_empty, T_actual=T_actual_2) #Capacity in dex at depth2 (maximum load factor)
    DWT_dc = design_capacity(T_design, T_empty, row['Length'], row['Width']) #Calculate the design capacity
    mod_capacity=DWT_dc*CI_1/CI_2 # Calculate the capacity for minimum operaional draft based on the design capacity of the vessel type
    payload_design = (1-consumable_design)*DWT_dc
    payload_lowwater = mod_capacity-consumable_lowwater*DWT_dc
    print(round(T_design,2),round(DWT_dc), round(payload_design),row['draft mod'], round(mod_capacity), round(payload_lowwater))

2.78 802 754 1.2 162 130
3.16 1713 1610 1.3 318 250
3.5 3066 2882 1.4 584 461
3.93 5499 5169 1.5 1083 863
4.22 7307 6868 1.5 1238 945


### 7.4 Dumb barges

In [14]:
dumbbarges = {'type': ['Barge','Barge','Barge'],
        'CEMT-class':  ['IV','V','V'],
        'Length': [70, 77, 90],
        'Width': [9.50, 11.40, 11.40],
        'draft design':[3.19, 3.98, 4.11], #Deep water desing draft
        'draft mod':[1.30, 1.40, 1.40]} # Minimum operational draft

dumbbarges_df = pd.DataFrame (dumbbarges, columns = ['type','CEMT-class','Length','Width','draft design', 'draft mod'])

print(dumbbarges_df)

    type CEMT-class  Length  Width  draft design  draft mod
0  Barge         IV      70    9.5          3.19        1.3
1  Barge          V      77   11.4          3.98        1.4
2  Barge          V      90   11.4          4.11        1.4


In [15]:
consumable_design=0.00
consumable_lowwater=0.00
ukc=0.2

for index, row in dumbbarges_df.iterrows():
    T_design = design_draft(row['type'], row['Length'], row['Width'])
    T_empty = empty_draft(row['type'], T_design, row['Length'], row['Width'])
    depth1=row['draft mod']+ukc
    depth2=T_design+ukc
    T_actual_1 = actual_draft(T_design, T_empty, depth=depth1, ukc=0.2)
    T_actual_2 = actual_draft(T_design, T_empty, depth=depth2, ukc=0.2)
    CI_1 = cap_index(T_empty, T_actual=T_actual_1) #Capacity index at depth1
    CI_2 = cap_index(T_empty, T_actual=T_actual_2) #Capacity in dex at depth2 (maximum load factor)
    DWT_dc = design_capacity(T_design, T_empty, row['Length'], row['Width']) #Calculate the design capacity
    mod_capacity=DWT_dc*CI_1/CI_2 # Calculate the capacity for minimum operaional draft based on the design capacity of the vessel type
    payload_design = (1-consumable_design)*DWT_dc
    payload_lowwater = mod_capacity-consumable_lowwater*DWT_dc
    print(round(T_design,2),round(DWT_dc), round(payload_design),row['draft mod'], round(mod_capacity), round(payload_lowwater))

3.19 1649 1649 1.3 472 472
3.98 2763 2763 1.4 604 604
4.11 3370 3370 1.4 716 716


General application by hand

In [16]:
# Input variables

vessel = "Dry_SH"
L = 110
B = 11.4
depth1 = 1.5
depth2 = 3.6

print(f"The vesseltype is {vessel} with dimensions {L}x{B} m")

#Calculate the design draft

T_design = design_draft(vessel, L, B)
print (f"The design draft is: {round(T_design, 2)} m")

#Caluclate the empty draft

T_empty = empty_draft(vessel, T_design, L, B)
print (f"The empty draft is: {round(T_empty, 2)} m")

#Calculate the actual draft

T_actual_1 = actual_draft(T_design, T_empty, depth=depth1, ukc=0.1)
T_actual_2 = actual_draft(T_design, T_empty, depth=depth2, ukc=0.1)

print (f"The actual draft for depth1 is: {round(T_actual_1, 2)} m")
print (f"The actual draft for depth2 is: {round(T_actual_2, 2)} m")

#Calculate the capacity index

CI_1 = cap_index(T_empty, T_actual=T_actual_1) #Capacity index at depth1
CI_2 = cap_index(T_empty, T_actual=T_actual_2) #Capacity in dex at depth2 (maximum load factor)

print (f"The capacity index for depth1 is: {round(CI_1, 2)}")
print (f"The capacity index for depth2 is: {round(CI_2, 2)}")

#Calculate the design capacity

DWT_dc = design_capacity(T_design, T_empty, L, B)
#print (f"The design capacity is: {round(DWT_dc, 0)} t")

# Calculate the capacity for minimum operaional draft based on the design capacity of the vessel type

mod_capacity=DWT_dc*CI_1/CI_2

print(f"The capacity for minimum operational draft is: {round(mod_capacity,0)} t")

#Define the payload at design conditions by subtracting an assumed percentage of the DWT for consumables: default = 6%
consumable_design=0.06
consumable_lowwater=0.04

payload_design = (1-consumable_design)*DWT_dc
payload_lowwater = mod_capacity-consumable_lowwater*DWT_dc

print (f"The design capacity is: {round(DWT_dc, 0)} t")
print(f"The payload for design draft is: {round(payload_design)} t")
print(f"The capacity for minimum operational draft is: {round(mod_capacity,0)} t")
print(f"The payload for minimum operational draft is: {round(payload_lowwater)} t")

The vesseltype is Dry_SH with dimensions 110x11.4 m
The design draft is: 3.44 m
The empty draft is: 0.76 m
The actual draft for depth1 is: 1.4 m
The actual draft for depth2 is: 3.44 m
The capacity index for depth1 is: 35.63
The capacity index for depth2 is: 156.72
The capacity for minimum operational draft is: 710.0 t
The design capacity is: 3125.0 t
The payload for design draft is: 2937 t
The capacity for minimum operational draft is: 710.0 t
The payload for minimum operational draft is: 585 t


In [17]:
# TODO: this variable was undefined, I now define it like this.
DWT = design_capacity(T_design, T_empty, L, B)


In [18]:
#Application of the capacity index to calculate the load factor

RCI = CI_2
print (f"The relative capacity index is: {round(RCI,2)}")

#Baseline capacity

BC = (DWT / RCI) * 100
print (f"The baseline capacity is: {round(BC, 2)} t")

#The absolute capacity at 1.6m (depth1):

AC = (BC * (CI_1 / 100))
print (f"The absolyte capacity is {round(AC, 2)} t")

#The load facor

lf = (AC / DWT)
print (f"The load factor at {depth1}m is: {round(lf, 2)}")

The relative capacity index is: 156.72
The baseline capacity is: 1993.78 t
The absolyte capacity is 710.29 t
The load factor at 1.5m is: 0.23


### 8. Comparison with linear interpolation 

In [19]:
vessel = "Dry"
L = 135
B = 11.40
depth1 = 2.4
depth2 = 3.7

T_design = 3.00 #Input variable in OpenCLSim model
T_empty = 1.00 #Input variable in OpenCLSim model


T_actual_1 = actual_draft(T_design, T_empty, depth1, ukc=0.25)
T_actual_2 = actual_draft(T_design, T_empty, depth2, ukc=0.25)

CI_1 = cap_index(T_empty, T_actual=T_actual_1)
CI_2 = cap_index(T_empty, T_actual=T_actual_2)

DWT = 3900 #Input variable in OpenCLSim model

#Baseline capacity
BC = DWT/RCI * 100

#Absolute capacity at 2.4m (depth1)
AC = (BC * (CI_1 / 100))

#Load factor

lf = (AC / (DWT))
print (f"Load facor: {round(lf, 2)}")

Load facor: 0.49


# CapacityIndex Class

In [20]:
Length = 110
Beam = 11.45

class CapacityIndex():
    
    def __init__(self, Length, Beam, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Length = Length
        self.Beam = Beam
        
        
    def design_draft(self, vessel):
        
        
        Tdesign_coefs = dict({"intercept":0,
                         "c1": 1.7244153371,
                         "c2": 2.2767179246,
                         "c3": 1.3365379898,
                         "c4": -5.9459308905,
                         "c5": 6.2902305560*10**-2,
                         "c6": 7.7398861528*10**-5,
                         "c7": 9.0052384439*10**-3,
                         "c8": 2.8438560877
                         })

    
        if vessel == "Container":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [1,0,0,0]

        elif vessel == "Dry":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,1,0,0]

        elif vessel == "Barge":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,1,0]

        elif vessel == "Tanker":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,0,1]

        T_design = Tdesign_coefs['intercept'] \
        + (dum_container * Tdesign_coefs['c1']) + (dum_dry * Tdesign_coefs['c2']) + (dum_barge * Tdesign_coefs['c3']) + (dum_tanker * Tdesign_coefs['c4']) \
        + (Tdesign_coefs['c5'] * dum_container * Length**0.4 * B**0.6) + (Tdesign_coefs['c6'] * dum_dry * Length**0.7 * B**2.6) \
        + (Tdesign_coefs['c7'] * dum_barge * Length**0.3 * B**1.8) + (Tdesign_coefs['c8'] * dum_tanker * Length**0.1 * B**0.3)
        
        return T_design
    
    def empty_draft(self, vessel, T_design):
    
        if vessel == "Container":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [1,0,0,0]

        elif vessel == "Dry":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,1,0,0]

        elif vessel == "Barge":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,1,0]

        elif vessel == "Tanker":
            [dum_container, dum_dry,
            dum_barge, dum_tanker] = [0,0,0,1]

        T_empty = Tempty_coefs['intercept'] \
        + (Tempty_coefs['c1'] * Beam) + (Tempty_coefs['c2'] * ((Length * T_design) / Beam)) + (Tempty_coefs['c3'] * (np.sqrt(Length * Beam))) \
        + (Tempty_coefs['c4'] * (Length * Beam * T_design)) + (Tempty_coefs['c5'] * dum_container) + (Tempty_coefs['c5'] * dum_dry) \
        + (Tempty_coefs['c6'] * dum_tanker) + (Tempty_coefs['c7'] * dum_barge)

        return T_empty
    
    def actual_draft(self, depth, ukc):
    
        if (T_design <= depth):

            T_actual = T_design

        elif T_empty > (depth -ukc):

            T_actual =  (f"No trip possible. Available depth smaller than empty draft: {depth - T_empty} m")

        elif (T_design > depth):

            T_actual = depth - ukc

        return T_actual
    
    def cap_index(self, T_empty, T_actual):
        
        CI = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2) \
        + (CI_coefs["c3"] * T_actual) + (CI_coefs["c4"] * T_actual**2) \
        + (CI_coefs["c5"] * (T_empty * T_actual))

        return CI
    
    def design_capacity(self, T_design, T_empty):
    
        DWT = capacity_coefs['intercept'] + (capacity_coefs['c1'] * Length * Beam * T_design) \
        + (capacity_coefs['c2'] * Length * Beam * T_empty)

        return DWT

In [21]:
Va = CapacityIndex(110, 11.45)

T_design = Va.design_draft("Container")
T_empty = Va.empty_draft("Container", T_design=T_design)
T_actual_1 = Va.actual_draft(depth=1.6, ukc=0.2)
T_actual_2 = Va.actual_draft(depth=3.7, ukc=0.2)

Capindex_1 = Va.cap_index(T_empty=T_empty, T_actual=T_actual_1)
Capindex_2 = Va.cap_index(T_empty=T_empty, T_actual=T_actual_2)
DWT = Va.design_capacity(T_design, T_empty)

In [22]:
#Baseline cap
BC = (DWT / Capindex_2) * 100
AC = BC * (Capindex_1/100)
AC
lf = AC/DWT
lf

0.19085094068807798